In [287]:
import pandas as pd
import numpy as np
import re

In [288]:
# pd.set_option('display.max_colwidth', None)

In [289]:
pd.reset_option('display.max_colwidth')

In [290]:
master_data = {
    "trojmiasto_2023_master" : "../original_data_2023/trojmiasto/ALL_DATA/gdansk_trojmiasto_master_data.json",
}

for key, value in master_data.items():
    master_data[key] = pd.read_json(value)  

trojmiasto_df = master_data["trojmiasto_2023_master"]
df = trojmiasto_df

In [291]:
df.head()

,link,rent,description,address,additional_info,Liczba pokoi,Piętro,Rok budowy,Jestem,Rodzaj nieruchomości,...,Liczba pięter w budynku,Powierzchnia,Kaucja,Typ ogrzewania,Niezależne pokoje,Dostępne od,Liczba osób w pokoju,Nr oferty,Nr licencji pośrednika,Powierzchnia działki
0,https://ogloszenia.trojmiasto.pl/nieruchomosci...,3 200 zł,"Apartament o pow. 40m2 , w pełni umeblowany i ...",GdańskPlater,[],2.0,Parter,1980.0,Firmą,Mieszkanie,...,5.0,40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://ogloszenia.trojmiasto.pl/nieruchomosci...,3 700 zł,"Komfortowe nowe mieszkanie, kameralny budynek,...",Gdańsk BrzeźnoHallera 232,[],3.0,1,2021.0,Osobą prywatną,Mieszkanie,...,3.0,45,3700.0,Centralne,NaN,NaN,NaN,NaN,NaN,NaN
2,https://ogloszenia.trojmiasto.pl/nieruchomosci...,3 000 zł,Mieszkanie w Brzeźnie przy ulicy PCK. W najbli...,Gdańsk BrzeźnoPolskiego Czerwonego Krzyża,[Wynajmę studentom],2.0,4,2023.0,Firmą,Mieszkanie,...,4.0,28,3000.0,NaN,Nie,Wrzesien/pazdziernik,NaN,NaN,NaN,NaN
3,https://ogloszenia.trojmiasto.pl/nieruchomosci...,3 200 zł,"ŚWIEŻO PO REMONCIE, KAPITALNA LOKALIZACJA. Do ...",Gdańsk MorenaCzecha,"[Wynajmę studentom, Balkon, Miejsce parkingowe...",2.0,2,1980.0,Firmą,Mieszkanie,...,4.0,50,4000.0,Miejskie,Nie,Zaraz,2-osobowy,NaN,NaN,NaN
4,https://ogloszenia.trojmiasto.pl/nieruchomosci...,2 200 zł,NAJEMCA NIE PŁACI KOSZTÓW PROWIZJI DOSTĘPNE OD...,Gdańsk Dolne MiastoŁąkowa,[],1.0,4,1910.0,Firmą,Mieszkanie,...,4.0,20,NaN,NaN,NaN,NaN,NaN,799981,NaN,NaN


In [292]:
df.shape

(6064, 21)

In [293]:
trojmiasto_heating_mapping = {
    'Centralne': 'Centralne',
    'Miejskie': 'Miejskie',
    'Co Miejskie': 'Centralno-miejskie',
    'Elektryczne': 'Elektryczne',
    'C. o. miejskie': 'Centralno-miejskie',
    'Co Własne': 'Centralne',
    '(Nieznane)': 'Inne',
    'Gazowe': 'Gazowe',
    'Ogrzewanie miejskie': 'Miejskie',
    'Kotłownia w bloku': 'Inne',
    'CO miejskie': 'Centralno-miejskie',
    'Kominek': 'Inne',
    'C/o': 'Centralne',
    'Cenyralne': 'Centralne',
    'C. o': 'Centralne',
    'Podłogowe wliczone w cenę': 'Inne',
    'Centralne, podłogowe': 'Centralne',
    'Gazowe, klimatyzator': 'Gazowe',
    'Centralne miejskie': 'Centralno-miejskie',
    'Centralny': 'Centralne',
    'Centralne, miejskie': 'Centralno-miejskie',
    'C. o.': 'Centralne',
    'C. O. miejskie': 'Centralno-miejskie',
    'Kaloryfery': 'Inne',
    'Centralne ogrzewanie': 'Centralne',
    'CO własne': 'Inne',
    'Centralne- z sieci': 'Centralno-miejskie',
    'GPEC': 'Inne',
    'Gaz': 'Gazowe',
    'Gazowy': 'Gazowe',
    'Miejskie (centralne)': 'Centralno-miejskie',
    'C. O.': 'Centralne',
    'Elektryczny': 'Elektryczne',
    'Gaz wspólnota': 'Gazowe',
    'Kotłowania': 'Inne',
    'Elektryczne akumulacyjne (II taryfa)': 'Elektryczne',
    'Ogrzrwanie centralne': 'Centralne',
    'Miejskie (gpc)': 'Miejskie',
    'OC': 'Centralne',
    'Miejski podłogowe': 'Miejskie',
    'Co miejskie': 'Centralno-miejskie',
    'Wentylacja wymuszona': 'Inne',
    'Eco Groszek': 'Inne'
}

trojmiasto_furnished_mapping = {
    'Tak': True,
    'Nie': False
}

trojmiasto_private_seller = {
    'Firmą': False,
    'Osobą prywatną': True
}

trojmiasto_dict_regions = {
    "Śródmieście": "Srodmiescie - Stare Miasto - Dolne Miasto - Długie Ogrody",
    "Stare": "Srodmiescie - Stare Miasto - Dolne Miasto - Długie Ogrody",
    "Starówka": "Srodmiescie - Stare Miasto - Dolne Miasto - Długie Ogrody",
    "Podwale": "Srodmiescie - Stare Miasto - Dolne Miasto - Długie Ogrody",
    "Sródmieście": "Srodmiescie - Stare Miasto - Dolne Miasto - Długie Ogrody",
    "Angielska": "Srodmiescie - Stare Miasto - Dolne Miasto - Długie Ogrody",
    "Długie": "Srodmiescie - Stare Miasto - Dolne Miasto - Długie Ogrody",
    
    "Przymorze": "Przymorze",
    "Przymorzeul.": "Przymorze",
    "Obrońców": "Przymorze",
    "Przymorze10": "Przymorze",
    
    "Zakoniczyn": "Chelm - Lostowice - Poludnie",
    "Łostowice": "Chelm - Lostowice - Poludnie",
    "Kowale": "Chelm - Lostowice - Poludnie",
    "Ujeścisko": "Chelm - Lostowice - Poludnie",
    "Południe": "Chelm - Lostowice - Poludnie",
    "Chełm": "Chelm - Lostowice - Poludnie",
    "Ujeścisko-": "Chelm - Lostowice - Poludnie",
    "Łostwice": "Chelm - Lostowice - Poludnie",
    "Antoniego": "Chelm - Lostowice - Poludnie",
    
    "Wrzeszcz": "Wrzeszcz",
    "Wrzwszcz": "Wrzeszcz",
    "Wrzeszczal": "Wrzeszcz",
    "Bora-": "Wrzeszcz",
    "Garnizon": "Wrzeszcz",
    
    "Brzeźno": "Brzezno",
    "Czarny": "Brzezno",
    
    "Żabianka": "Zabianka - Wejhera - Jelitkowo - Tysiaclecia",
    "Jelitkowo": "Zabianka - Wejhera - Jelitkowo - Tysiaclecia",
    "Jelitkowski": "Zabianka - Wejhera - Jelitkowo - Tysiaclecia",
    
    "Aniołki": "Siedlce - Aniolki - Mlyniska - Mickiewicza",
    "Siedle": "Siedlce - Aniolki - Mlyniska - Mickiewicza",
    "Cygańska": "Siedlce - Aniolki - Mlyniska - Mickiewicza",
    "Wzgórze": "Siedlce - Aniolki - Mlyniska - Mickiewicza",
    
    "Jasień": "Jasien - Matarnia - Kokoszki - Osowa",
    "Rebiechowo": "Jasien - Matarnia - Kokoszki - Osowa",
    "Osowa": "Jasien - Matarnia - Kokoszki - Osowa",
    "Matarnia": "Jasien - Matarnia - Kokoszki - Osowa",
    "Kokoszki": "Jasien - Matarnia - Kokoszki - Osowa",
    "Karczemki": "Jasien - Matarnia - Kokoszki - Osowa",
    "Kiełpinek": "Jasien - Matarnia - Kokoszki - Osowa",
    
    "Zaspa": "Zaspa",
    
    "Morena": "Piecki Migowo - Suchanino - Morena",
    "Suchanino": "Piecki Migowo - Suchanino - Morena",
    "Piecki": "Piecki Migowo - Suchanino - Morena",
    "Piecki-": "Piecki Migowo - Suchanino - Morena",
    "Suchaninovan": "Piecki Migowo - Suchanino - Morena",
    "Morenagen": "Piecki Migowo - Suchanino - Morena",
    "Morena/": "Piecki Migowo - Suchanino - Morena",
    
    "Letnica": "Letnica",
    "Letnica37": "Letnica",
    
    "Oliwa": "Oliwa - Strzyza",
    "Strzyża": "Oliwa - Strzyza",
    
    "Maćkowy": "Orunia",
    
    "Stogi": "Przerobka - Stogi - Nowy Port",
    "Siedlce": "Przerobka - Stogi - Nowy Port",
    "Przeróbka": "Przerobka - Stogi - Nowy Port",
}


trojmasto_level_mapping = {
    'Parter': 0,
    '1': 1, '2': 2, '3':3, '4':4, '5':5, '6':6, '7':7, '8':8, '9':9, '10':10,
    '11': 11,
    '12': 11,
    '13': 11,
    '14': 11,
    '15': 11,
    '16': 11,
    '17': 11,
    'poddasze': 11,
    1001: 1001
}

trojmasto_rooms_mapping = {
    2.0 : 2.0,
    1.0 : 1.0,
    3.0 : 3.0,
    4.0 : 4.0,
    5.0 : 4.0
}

In [294]:
dirty_data = df.copy()

In [295]:
df.isna().sum()

link                          0
rent                          0
description                   0
address                       0
additional_info               0
Liczba pokoi                  3
Piętro                       20
Rok budowy                  608
Jestem                        1
Rodzaj nieruchomości          1
Umeblowane                   25
Liczba pięter w budynku     105
Powierzchnia                  1
Kaucja                     3118
Typ ogrzewania             2218
Niezależne pokoje          4461
Dostępne od                3198
Liczba osób w pokoju       5078
Nr oferty                  4099
Nr licencji pośrednika     5575
Powierzchnia działki       5954
dtype: int64

In [296]:
df.columns

Index(['link', 'rent', 'description', 'address', 'additional_info',
       'Liczba pokoi', 'Piętro', 'Rok budowy', 'Jestem',
       'Rodzaj nieruchomości', 'Umeblowane', 'Liczba pięter w budynku',
       'Powierzchnia', 'Kaucja', 'Typ ogrzewania', 'Niezależne pokoje',
       'Dostępne od', 'Liczba osób w pokoju', 'Nr oferty',
       'Nr licencji pośrednika', 'Powierzchnia działki'],
      dtype='object')

In [297]:
def fill_na_with_mean(column_name: str, data_frame: pd.DataFrame):
    mean = data_frame[column_name][data_frame[column_name].notna()].astype(int).mean()
    data_frame.loc[:, column_name] = data_frame[column_name].fillna(mean)
    data_frame.loc[:, column_name] = data_frame[column_name].astype(int)
    return data_frame

def extract_first_word(given_string: str):
    big_letter = 0
    for idx, letter in enumerate(given_string):
        if letter.isupper() and big_letter == 1:
            return given_string[:idx]
        elif letter.isupper():
            big_letter += 1
    return given_string

def clear_trojmiasto(df):
    
    df = df.copy()
    
    # columns_to_drop
    df.drop(
        labels=[
                'description', 
                'Dostępne od', 
                'Nr licencji pośrednika', 
                'Powierzchnia działki', 
                'Nr oferty',
                'Niezależne pokoje',
                'Liczba osób w pokoju',
                'Kaucja',
                'Rodzaj nieruchomości',
                'Typ ogrzewania',
                'link',
                'additional_info',
                'Rok budowy',
                'Liczba pięter w budynku'
        ],
                axis=1, inplace=True)
    
    # rent variable
    df['rent'] = df['rent'].apply(lambda x: float(x.replace("zł", "").replace(",",".")
                                                  .replace(" ","").replace("N/A", "0")))

    # level
    df = df.rename(mapper={"Piętro": "level"}, axis=1)
    df['level'] = df['level'].fillna(1001)
    df['level'] = df['level'].map(trojmasto_level_mapping)
    
    # rooms
    df.rename(mapper={"Liczba pokoi": "rooms"}, axis=1, inplace=True)
    df = df[df['rooms'] <= 5]
    df["rooms"] = df["rooms"].map(trojmasto_rooms_mapping)
    
#     # build_year
#     df = df[df['Rok budowy'] >= 1850]
#     df = fill_na_with_mean(column_name="Rok budowy", data_frame=df)
#     df.rename(mapper={'Rok budowy': "build_year"}, axis=1, inplace=True)

#     # heating_type
#     df['Typ ogrzewania'] = df['Typ ogrzewania'].map(heating_mapping)
#     df['Typ ogrzewania'].fillna('Inne', axis=0, inplace=True)
#     df.rename(mapper={'Typ ogrzewania': "heating_type"}, axis=1, inplace=True)

    # furnished
    df.rename(mapper={'Umeblowane': 'furnished'}, axis=1, inplace=True)
    df['furnished'] = df['furnished'].map(trojmiasto_furnished_mapping)
    df['furnished'].fillna(True, axis=0, inplace=True)

    # surface
    df.rename(mapper={'Powierzchnia': 'surface'}, axis=1, inplace=True)
    df['surface'].str.replace(',','.').str.replace(" ", "").astype(float)

    # private seller
    df.rename(mapper={'Jestem': 'private_seller'}, axis=1, inplace=True)
    df['private_seller'] = df['private_seller'].map(trojmiasto_private_seller)

#     # address
    df['address2'] = df['address'].str.strip().str.replace(" ","").str.replace("Gdańsk", "").str.replace("gdańsk", "").str.lstrip("\xa0").apply(extract_first_word)
    df['address2'] = df['address2'].map(trojmiasto_dict_regions)
    return df

In [298]:
df_dirty = dirty_data.copy()

In [299]:
df = clear_trojmiasto(df_dirty)

In [300]:
df.head()

,rent,address,rooms,level,private_seller,furnished,surface,address2
0,3200.0,GdańskPlater,2.0,0,False,True,40,NaN
1,3700.0,Gdańsk BrzeźnoHallera 232,3.0,1,True,True,45,Brzezno
2,3000.0,Gdańsk BrzeźnoPolskiego Czerwonego Krzyża,2.0,4,False,True,28,Brzezno
3,3200.0,Gdańsk MorenaCzecha,2.0,2,False,True,50,Piecki Migowo - Suchanino - Morena
4,2200.0,Gdańsk Dolne MiastoŁąkowa,1.0,4,False,True,20,NaN


In [302]:
df[df["address2"].isna()].value_counts()

Series([], Name: count, dtype: int64)

In [266]:
df["address"].str.lower().str.replace("gdańsk", "").value_counts()

address
 przymorzeobrońców wybrzeża     107
 przymorześląska                103
 długie ogrodyszafarnia          96
 śródmieście                     90
 wrzeszcz                        70
                               ... 
 śródmieściemalczewskiego 78      1
 śródmieściedługie ogody          1
 siedlcegoszczyńskiego            1
 jasieńleskiego 26                1
 osowaperseusza                   1
Name: count, Length: 1637, dtype: int64

In [268]:
df["address"].str.replace("Gdańsk", "").value_counts()

address
 PrzymorzeObrońców Wybrzeża              107
 PrzymorzeŚląska                         103
 Długie OgrodySzafarnia                   96
 Śródmieście                              90
 Wrzeszcz                                 68
                                        ... 
 ŚródmieścieŁąkowa 60D/10                  1
 ŚródmieściePodwale Staromiejskie 108      1
 Brzeźnoa 18                               1
Leskiego 34                                1
 OsowaPerseusza                            1
Name: count, Length: 1669, dtype: int64

In [269]:
df[df['address2'].isna()]['address'].value_counts()

KeyError: 'address2'

In [ ]:
df['address2'].str.split("k")[1:]

In [ ]:
df['address2'].isna().sum()

In [270]:
district = re.findall('[A-Z][^A-Z]*', str(df['address'].str.replace("Gdańsk", "").str.replace(" ","")))

In [282]:
df['address_tied'] = df['address'].str.replace("Gdańsk", "").str.replace(" ","").str.lstrip("\xa0").str.replace("/", "").apply(extract_first_word)

In [283]:
df['address_tied'].value_counts()

address_tied
Śródmieście               906
Przymorze                 824
Wrzeszcz                  674
Długie                    283
Brzeźno                   250
                         ... 
wrzeszczstaszica3           1
wrzeszczManifestu           1
Jasieńlawendowewzgorze      1
Letnica37                   1
Żabiankaorłowska9c          1
Name: count, Length: 253, dtype: int64

In [284]:
df['address_tied'].map(dict_regions).value_counts()

address_tied
Śródmieście    1423
Przymorze       839
Wrzeszcz        730
Łostowice       282
Brzeźno         272
Morena          269
Jasień          208
Zaspa           193
Letnica         179
Oliwa           166
Żabianka        161
Siedlce         124
Chełm           107
Jelitkowo       106
Stogi           102
Południe         98
Orunia           89
Matarnia         74
Osowa            67
Aniołki          64
Name: count, dtype: int64

In [ ]:
dict_count = {}

for k, v in dict_regions.items():
    if v not in dict_count.keys():
        dict_count[v] = 1
    if v in dict_count.keys():
        dict_count[v] += 1

In [273]:
dict_regions = {
    "Śródmieście": "Srodmiescie - Stare Miasto - Dolne Miasto - Długie Ogrody",
    "Stare": "Srodmiescie - Stare Miasto - Dolne Miasto - Długie Ogrody",
    "Starówka": "Srodmiescie - Stare Miasto - Dolne Miasto - Długie Ogrody",
    "Podwale": "Srodmiescie - Stare Miasto - Dolne Miasto - Długie Ogrody",
    "Sródmieście": "Srodmiescie - Stare Miasto - Dolne Miasto - Długie Ogrody",
    "Angielska": "Srodmiescie - Stare Miasto - Dolne Miasto - Długie Ogrody",
    "Długie": "Srodmiescie - Stare Miasto - Dolne Miasto - Długie Ogrody",
    
    "Przymorze": "Przymorze",
    "Przymorzeul.": "Przymorze",
    "Obrońców": "Przymorze",
    "Przymorze10": "Przymorze",
    
    "Zakoniczyn": "Chelm - Lostowice - Poludnie",
    "Łostowice": "Chelm - Lostowice - Poludnie",
    "Kowale": "Chelm - Lostowice - Poludnie",
    "Ujeścisko": "Chelm - Lostowice - Poludnie",
    "Południe": "Chelm - Lostowice - Poludnie",
    "Chełm": "Chelm - Lostowice - Poludnie",
    "Ujeścisko-": "Chelm - Lostowice - Poludnie",
    "Łostwice": "Chelm - Lostowice - Poludnie",
    "Antoniego": "Chelm - Lostowice - Poludnie",
    
    "Wrzeszcz": "Wrzeszcz",
    "Wrzwszcz": "Wrzeszcz",
    "Wrzeszczal": "Wrzeszcz",
    "Bora-": "Wrzeszcz",
    "Garnizon": "Wrzeszcz",
    
    "Brzeźno": "Brzezno",
    "Czarny": "Brzezno",
    
    "Żabianka": "Zabianka - Wejhera - Jelitkowo - Tysiaclecia",
    "Jelitkowo": "Zabianka - Wejhera - Jelitkowo - Tysiaclecia",
    "Jelitkowski": "Zabianka - Wejhera - Jelitkowo - Tysiaclecia",
    
    "Aniołki": "Siedlce - Aniolki - Mlyniska - Mickiewicza",
    "Siedle": "Siedlce - Aniolki - Mlyniska - Mickiewicza",
    "Cygańska": "Siedlce - Aniolki - Mlyniska - Mickiewicza",
    "Wzgórze": "Siedlce - Aniolki - Mlyniska - Mickiewicza",
    
    "Jasień": "Jasien - Matarnia - Kokoszki - Osowa",
    "Rebiechowo": "Jasien - Matarnia - Kokoszki - Osowa",
    "Osowa": "Jasien - Matarnia - Kokoszki - Osowa",
    "Matarnia": "Jasien - Matarnia - Kokoszki - Osowa",
    "Kokoszki": "Jasien - Matarnia - Kokoszki - Osowa",
    "Karczemki": "Jasien - Matarnia - Kokoszki - Osowa",
    "Kiełpinek": "Jasien - Matarnia - Kokoszki - Osowa",
    
    "Zaspa": "Zaspa",
    
    "Morena": "Piecki Migowo - Suchanino - Morena",
    "Suchanino": "Piecki Migowo - Suchanino - Morena",
    "Piecki": "Piecki Migowo - Suchanino - Morena",
    "Piecki-": "Piecki Migowo - Suchanino - Morena",
    "Suchaninovan": "Piecki Migowo - Suchanino - Morena",
    "Morenagen": "Piecki Migowo - Suchanino - Morena",
    "Morena/": "Piecki Migowo - Suchanino - Morena",
    
    "Letnica": "Letnica",
    "Letnica37": "Letnica",
    
    "Oliwa": "Oliwa - Strzyza",
    "Strzyża": "Oliwa - Strzyza",
    
    "Maćkowy": "Orunia",
    
    "Stogi": "Przerobka - Stogi - Nowy Port",
    "Siedlce": "Przerobka - Stogi - Nowy Port",
    "Przeróbka": "Przerobka - Stogi - Nowy Port",
}

In [ ]:
trojmiasto_gda_df.isna().sum()

In [ ]:
trojmiasto_gda_df.columns

In [ ]:
trojmiasto_gda_df['Jestem'].unique()


In [ ]:
trojmiasto_gda_df['rent']

In [ ]:
trojmiasto_gda_df.columns